<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./03_WEKEO_dias_service.ipynb"><< 03 - WEkEO - Copernicus Data and Information Access Service</a><span style="float:right;"><a href="./05_google_earth_engine.ipynb">05 - Google Earth Engine>></a></span>

# Amazon Web Services - Open Data Registry

### About

[Earth on AWS]("https://aws.amazon.com/earth/") is a registry of open geospatial datasets on Amazon Web Services.
* [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) is the Amazon Web Services (AWS) SDK for Python in order e.g. to access data on AWS S3 storage



### Data

### How to retrieve data?

<hr>

#### Load required libraries

In [ ]:
import os
import sys
import json
import time
import base64

import boto3
import botocore


import requests
import warnings
warnings.filterwarnings('ignore')

#### Load helper functions

In [ ]:
from ipynb.fs.full.hda_api_functions import *

<hr>

In [ ]:
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))
s5p_bucket = 'meeo-s5p'

In [ ]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=s5p_bucket, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

In [ ]:
prefix=('COGT/OFFL/L3__NO2__')
paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=s5p_bucket, Prefix=prefix)


In [ ]:
import re
#regex = re.compile('*column_4326.tif')
selected_files = list(filter(lambda x: x.endswith('mask75_4326.tif'), key_list))
len(selected_files)

In [ ]:
i=0
key_list=[]
for page in pages:
    for obj in page['Contents']:
        print(i)
        print(obj.get('Key'))
        key_list.append(obj.get('Key'))
        i=i+1

In [ ]:
for i in selected_files[439:443]:
    print(i)
    tmp = i.split('/')
    print(tmp[5])
    client.download_file(s5p_bucket, i, '../eodata/sentinel5p/no2/'+tmp[5])

<br>

<a href="./03_WEKEO_dias_service.ipynb"><< 03 - WEkEO - Copernicus Data and Information Access Service</a><span style="float:right;"><a href="./05_google_earth_engine.ipynb">05 - Google Earth Engine>></a></span><br>
<a href="./00_index.ipynb"><< Index</a>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>